## Problem

下表に示す 3 種類の食品 (a, b, c) を使って、2 種類の栄養素 (x, y) の摂取量を満たす一番安い組み合わせを求めてください。

|         | 食品a | 食品b |  食品c | 摂取量 |
|      -  |     - |     - |      - |      - |
| 栄養素x |     3 |     1 |      2 |     15 |
| 栄養素y |     1 |     2 |      4 |     10 |
|    単価 |     4 |     2 |      5 |        |

(http://www.nct9.ne.jp/m_hiroi/light/pulp01.html#chap03)

In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
    columns=['食品a', '食品b', '食品c', '摂取量'],
    index=['栄養素x', '栄養素y', '単価'],
    data=[
        [3, 1, 2, 15],
        [1, 2, 4, 10],
        [4, 2, 5]
        ]
    )
df

,食品a,食品b,食品c,摂取量
栄養素x,3,1,2,15.0
栄養素y,1,2,4,10.0
単価,4,2,5,NaN


In [3]:
problem = pulp.LpProblem(name="diet", sense=pulp.LpMinimize)
problem

diet:
MINIMIZE
None
VARIABLES

In [4]:
food = pulp.LpVariable.dicts("", df.columns[:-1], lowBound=0, cat=pulp.LpInteger)
food

{'食品a': _食品a, '食品b': _食品b, '食品c': _食品c}

In [5]:
problem += pulp.lpSum([food[i] * df[i]["単価"] for i in food.keys()])
problem

diet:
MINIMIZE
4*_食品a + 2*_食品b + 5*_食品c + 0
VARIABLES
0 <= _食品a Integer
0 <= _食品b Integer
0 <= _食品c Integer

In [6]:
for j in df.index[:-1]:
    problem += pulp.lpSum([food[i] * df[i][j] for i in food.keys()]) >= df["摂取量"][j]
problem

diet:
MINIMIZE
4*_食品a + 2*_食品b + 5*_食品c + 0
SUBJECT TO
_C1: 3 _食品a + _食品b + 2 _食品c >= 15

_C2: _食品a + 2 _食品b + 4 _食品c >= 10

VARIABLES
0 <= _食品a Integer
0 <= _食品b Integer
0 <= _食品c Integer

In [7]:
result = problem.solve()
print("Status", pulp.LpStatus[result])

Status Optimal


In [8]:
print("=" * 3, "Result", "=" * 29)
[print(i, ":", food[i].value() ) for i in food.keys()]
print('総価格: {}'.format(pulp.value(problem.objective)))
print("=" * 40)

=== Result =============================
食品a : 4.0
食品b : 3.0
食品c : 0.0
総価格: 22.0
